In [1]:
import pandas as pd
import nibabel as nib
import numpy as np
import os
import ast

# 1. Load and clean data
df = pd.read_csv('merged_brain_age_hdr.csv', converters={'hdr_paths': ast.literal_eval})
df['hdr_paths'] = df['hdr_paths'].apply(lambda x: [p.strip() for p in x if p.strip()])
df['selected_hdr'] = df['hdr_paths'].apply(lambda x: x[1] if len(x)>=2 else None)
df = df.dropna(subset=['selected_hdr']).reset_index(drop=True)

# 2. Convert to absolute paths
df['selected_hdr'] = df['selected_hdr'].apply(os.path.abspath)

# 3. Validate paths
df = df[df['selected_hdr'].apply(os.path.exists)].reset_index(drop=True)

# 4. Verify sample file
if df.empty:
    print("DataFrame is empty after filtering! Check file paths.")
else:
    sample_path = df['selected_hdr'].iloc[0]
    print(f"Sample path: {sample_path}")
    print(f"File exists: {os.path.exists(sample_path)}")

try:
    sample_img = nib.load(sample_path)
    print(f"Image shape: {sample_img.header.get_data_shape()}")
except Exception as e:
    print(f"Error loading sample: {str(e)}")

Sample path: E:\Brain Age prediction\Brain Age prediction\disc1\OAS1_0001_MR1\processed\MPRAGE\T88_111\OAS1_0001_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr
File exists: True
Image shape: (176, 208, 176, 1)


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader 
import torchio as tio


class BrainAgeDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file, converters={'hdr_paths': ast.literal_eval})
        self.data['hdr_paths'] = self.data['hdr_paths'].apply(lambda x: [p.strip() for p in x if p.strip()])
        self.data['selected_hdr'] = self.data['hdr_paths'].apply(lambda x: x[1] if len(x) >= 2 else None)
        self.data = self.data.dropna(subset=['selected_hdr']).reset_index(drop=True)
        self.data['selected_hdr'] = self.data['selected_hdr'].apply(os.path.abspath)
        self.data = self.data[self.data['selected_hdr'].apply(os.path.exists)].reset_index(drop=True)

        # Assign the transform correctly
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        hdr_path = self.data.loc[idx, 'selected_hdr']
        age = self.data.loc[idx, 'Age']
    
        img, age = self.load_3d_volume(hdr_path, age)
    
        if img is None:
            raise ValueError(f"Error loading image at index {idx}")
    
        #print(f"Initial image shape: {img.shape}")  # (D, H, W, C)
    
        # Ensure the shape is (D, H, W, C)
        if img.ndim == 3:  # If missing channel dimension
            img = np.expand_dims(img, -1)
    
        #print(f"Shape after adding channel: {img.shape}")  # (D, H, W, C)
    
        # Apply augmentation if a transform is provided
        if self.transform:
            img_tensor = torch.tensor(img).permute(3, 0, 1, 2)  # (D, H, W, C) -> (C, D, H, W)
            #print(f"Shape before augmentation: {img_tensor.shape}")  # (C, D, H, W)
            img = self.transform(tio.ScalarImage(tensor=img_tensor)).tensor.numpy()
            #print(f"Shape after augmentation: {img.shape}")  # (C, D, H, W)
    
        # Adjust the final shape to (C, D, H, W)
        img = torch.tensor(img).permute(0, 1, 2, 3)  # Keep the original (C, D, H, W)
    
        #print(f"Final shape before returning: {img.shape}")  # (1, D, H, W)
    
        return img, torch.tensor(age, dtype=torch.float32)
    
        
    def load_3d_volume(self, hdr_path, age):
        try:
            img = nib.load(hdr_path).get_fdata()

            # Normalize to [0, 1]
            min_val, max_val = img.min(), img.max()
            img = (img - min_val) / (max_val - min_val + 1e-8)

            if img.ndim == 3:  # (176, 208, 176) -> (176, 208, 176, 1)
                img = np.expand_dims(img, -1)

            return img.astype(np.float32), np.array(age, dtype=np.float32)

        except Exception as e:
            print(f"Error loading {hdr_path}: {e}")
            return None, None

C:\Users\Siddhant Jaiswal\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Define augmentations using TorchIO
augmentations = tio.Compose([
    tio.RandomAffine(scales=(1, 1), degrees=(0, 0, 15), translation=0),
    tio.RandomBlur(p=0.3),
    tio.RandomNoise(p=0.3)
])

# Datasets and DataLoader
from torch.utils.data import random_split, DataLoader

# Load full dataset
full_dataset = BrainAgeDataset(csv_file='merged_brain_age_hdr.csv', transform=augmentations)

# Define split ratio (e.g., 80% train, 20% validation)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Split dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True, num_workers=0)  # Fixed typo 'shuffel' -> 'shuffle'


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Shallow3DCNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=1):
        super(Shallow3DCNN, self).__init__()

        self.conv1 = nn.Conv3d(in_channels, 16, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm3d(16)
        self.pool1 = nn.MaxPool3d(kernel_size=2)  # (88, 104, 88)

        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm3d(32)
        self.pool2 = nn.MaxPool3d(kernel_size=2)  # (44, 52, 44)

        self.dropout = nn.Dropout(0.3)

        
        flattened_size = 32 * 44 * 52 * 44

        self.fc1 = nn.Linear(flattened_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)

        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [16]:
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold

def cross_validate_model(model_class, X, y, k=5, epochs=10, batch_size=4, lr=1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    dataset = TensorDataset(X_tensor, y_tensor)
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)

    all_fold_losses = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"\nFold {fold + 1}")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size)

        model = model_class().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = nn.MSELoss()

        for epoch in range(epochs):
            model.train()
            train_loss = 0
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    val_loss += loss.item()

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss/len(train_loader):.4f} - Val Loss: {val_loss/len(val_loader):.4f}")

        all_fold_losses.append(val_loss / len(val_loader))

    print("\nCross-validation complete.")
    print(f"Avg Validation Loss across folds: {np.mean(all_fold_losses):.4f}")
    return all_fold_losses, all_val_preds

In [17]:
num_samples = 50  
X_dummy = np.random.rand(num_samples, 1, 176, 208, 176).astype(np.float32)
y_dummy = np.random.rand(num_samples).astype(np.float32)



In [18]:
history, preds = cross_validate_model(Shallow3DCNN, X_dummy, y_dummy, k=5, epochs=10, batch_size=4)


for fold_idx, (train_loss, val_loss) in enumerate(history):
    plt.plot(train_loss, label=f'Train Fold {fold_idx+1}')
    plt.plot(val_loss, label=f'Val Fold {fold_idx+1}', linestyle='--')

plt.title("Training & Validation Loss per Fold")
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Using device: cpu

Fold 1
Epoch 1/10 - Train Loss: 94165.0443 - Val Loss: 185.4079
Epoch 2/10 - Train Loss: 1055.6514 - Val Loss: 75.2396
Epoch 3/10 - Train Loss: 133.8908 - Val Loss: 5.4091
Epoch 4/10 - Train Loss: 44.3650 - Val Loss: 29.6204
Epoch 5/10 - Train Loss: 42.9800 - Val Loss: 4.6057
Epoch 6/10 - Train Loss: 13.6528 - Val Loss: 9.8316
Epoch 7/10 - Train Loss: 9.8022 - Val Loss: 2.0129
Epoch 8/10 - Train Loss: 6.3097 - Val Loss: 0.4705
Epoch 9/10 - Train Loss: 4.7990 - Val Loss: 5.2699
Epoch 10/10 - Train Loss: 7.0774 - Val Loss: 1.4769

Fold 2
Epoch 1/10 - Train Loss: 90285.0504 - Val Loss: 1.0710
Epoch 2/10 - Train Loss: 26.6662 - Val Loss: 0.6093
Epoch 3/10 - Train Loss: 19.1680 - Val Loss: 3.7987
Epoch 4/10 - Train Loss: 33.0224 - Val Loss: 0.8391
Epoch 5/10 - Train Loss: 30.1966 - Val Loss: 20.0306
Epoch 6/10 - Train Loss: 25.1098 - Val Loss: 44.3157
Epoch 7/10 - Train Loss: 62.1123 - Val Loss: 38.7718
Epoch 8/10 - Train Loss: 93.4420 - Val Loss: 37.4342
Epoch 9/10 - Tra

NameError: name 'all_val_preds' is not defined